# Spatial and temporal stability of the bank vole gut microbiota in a disturbed environment


## Abstract

Gut microbiota play an important role in host health. Yet, the drivers and patterns of microbiota imbalance (dysbiosis) in wild animals remain largely unexplored. One hypothesised outcome of stress on animal microbiomes is a destabilised microbiota community that is characterised by an increase in inter-individual differences (i.e. higher beta diversity) compared with microbiomes of healthy animals, which are expected to be temporally stable and relatively similar among individuals (i.e. lower beta diversity). This set of predictions for response of microbiomes to stressors is known as the Anna Karenina principle (AKP) for animal microbiomes. We examine the AKP in a wild mammal experiencing environment stress by conducting a capture-mark-recapture survey of bank voles (*Myodes glareolus*) inhabiting areas that contrast in levels of radionuclide contamination (in Chernobyl, Ukraine). Counter to key predictions of the AKP, bank voles that are not exposed to radionuclides harbour diverse (high beta diversity) and temporally dynamic gut microbiota communities, presumably tracking the natural spatio-temporal variation in resources. Conversely, bank voles exposed to radionuclides host more similar gut microbiota communities that are temporally stable, potentially due to a dysbiosis or selection (on host or bacteria) imposed by chronic radiation exposure. The implication of these data is that environmental stress (radiation exposure) can constrain the natural spatial and temporal diversity of wild animal gut microbiota.

### Analysis Notebook

This notebook contains commands to process raw read data and reproduce the qiime2 analyses. This [repository](https://github.com/alavrinienko/chern-cmr-voles) provides the main data files, *i.e.* metadata and dada2 feature-tables, from which the rest of the data can be generated using the commands below. The raw sequence data are freely available through Qiita study ID 12325 or EBI-European Nucleotide Archive: [ERP114357](https://www.ebi.ac.uk/ena/data/view/ERP114357). To analyse the CMR bank voles microbiome data we used computing resources and infrastructure provided by [the Finnish Centre for Scientific Computing (CSC)](https://www.csc.fi/csc).

### Requirements

qiime2 is required to run most of the commands described below (the 2018.2 qiime2 version was used). To install qiime2 follow these instructions: https://docs.qiime2.org/2019.1/install/. To activate the qiime2 conda environment run: source activate qiime2-2019.1 (or the other version, *i.e.* 2018.2).

## data import

The total data set contains 28 bank vole individuals, sampled twice ('first' & 'second', the total n=56 samples) over the capture-mark-recapture experiment. The 250PE read data were generated using the Illumina MiSeq platform. First, we need to import sequencing data to use with the qiime2 via a manifest file. This will inform qiime about samples id's, full path and sequences direction (forward,reverse). See importing tutorials for the 'fastq manifest formats' [here](https://docs.qiime2.org/2018.2/tutorials/importing/#fastq-manifest-formats).

In [ ]:
!qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path males_rt_manifest.txt \
    --source-format PairedEndFastqManifestPhred33 \
    --output-path malesrt_demux_paired-end.qza

The fastq files were successfully imported into the working directory. We would want to summarize and visualize the data (imported into qiime2 *.qza* file).

In [ ]:
!qiime demux summarize \
--i-data malesrt_demux_paired-end.qza \
--o-visualization malesrt_demux.qzv

Examine the generated *.qzv* file in [qiime2 viewer](https://view.qiime2.org). Take a note on the sequence counts (amount of reads) per sample, how many total reads are available to qiime2 in the CMR data set, also check which samples have the lowest (and how many) number of reads and what is the mean/median amount of reads per sample.

Then, examine the interactive quality plots, they display the reads quality in defined position according to the Phred33 qc format. The median quality score for both forward and reverse reads keeps over 36-37, however we also observe some quality drop in the end of the reverse reads. To ensure excellent data for the downstream analysis, we can leave forward as it is, but apply *truncation* for reverse reads at position 166, as in this case, the lowest quality at the bottom of the box will be 26.

Of important note, these data are free of adapters and primers. These, including any other sequencing constucts were removed as a part of the sequencing facility service, so no additional trimming from the left needed.

## denoising and the amplicon sequence variants (ASVs) picking using dada2

We will use the [dada2](https://docs.qiime2.org/2018.2/plugins/available/dada2/denoise-paired/) implemented in qiime2 for denoising and the amplicon sequence variants (ASVs) picking. We will adjust trimming settings within the dada2 based on the information that we got from examining the quality plots generated above.

Alternatively, continue from the metadata-related commands and examine the dada2 feature-table and the representative sequences that were used in the original analyses for the manuscript preparation. These files can be found in [this repository at GitHub](https://github.com/alavrinienko/chern-cmr-voles).

In [ ]:
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs malesrt_demux_paired-end.qza \
    --p-trim-left-f 0 \
    --p-trunc-len-f 250 \
    --p-trim-left-r 0 \
    --p-trunc-len-r 166 \
    --p-n-threads 0 \
    --o-representative-sequences dada2_malesrt/malesrt2_rep-seqs-dada2.qza \
    --o-table dada2_malesrt/malesrt2_table-dada2.qza

In [ ]:
!qiime feature-table summarize \
  --i-table dada2_malesrt/malesrt2_table-dada2.qza \
  --o-visualization dada2_malesrt/malesrt2_table-dada2.qzv \
  --m-sample-metadata-file metadata_malesrt.txt

## metadata

All the metadata associated with the samples in this study can be found [here](https://github.com/alavrinienko/chern-cmr-voles).

In [ ]:
!qiime metadata tabulate \
 --m-input-file metadata_malesrt.txt \
 --o-visualization metadata_malesrt.qzv

In [ ]:
#add metadata and examine dada2 feature-table
!qiime feature-table summarize \
  --i-table dada2_malesrt/malesrt2_table-dada2.qza \
  --o-visualization dada2_malesrt/malesrt2_table-dada2.qzv \
  --m-sample-metadata-file metadata_malesrt.txt

#examine representative sequences
!qiime feature-table tabulate-seqs \
 --i-data dada2_malesrt/malesrt2_rep-seqs-dada2.qza \
 --o-visualization dada2_malesrt/malesrt2_rep-seqs-dada2.qzv

## phylogenetics

Construct a phylogenetic tree using FastTree, and use the longest branch to root the tree (‘midrooting’).

In [ ]:
#first, align the representative sequences
!qiime alignment mafft \
  --i-sequences dada2_malesrt/malesrt2_rep-seqs-dada2.qza \
  --o-alignment dada2_malesrt/aligned-malesrt2_rep-seqs-dada2.qza

#mask phyloginetically unimportant sites
!qiime alignment mask \
  --i-alignment dada2_malesrt/aligned-malesrt2_rep-seqs-dada2.qza \
  --o-masked-alignment dada2_malesrt/masked-aligned-malesrt2_rep-seqs-dada2.qza

#create a tree
!qiime phylogeny fasttree \
  --i-alignment dada2_malesrt/masked-aligned-malesrt2_rep-seqs-dada2.qza \
  --o-tree dada2_malesrt/unrooted-malesrt2_tree_dada2.qza

#root generated tree at midpoint
!qiime phylogeny midpoint-root \
  --i-tree dada2_malesrt/unrooted-malesrt2_tree_dada2.qza \
  --o-rooted-tree dada2_malesrt/rooted-malesrt2_tree_dada2.qza

## feature-table filtering

Remove low-abundance features from the feature-table (features with the frequency <10 across all samples). It is important to filter out potential errors/artifacts that were retained during the denoising step, thus needs to be done before the downstream core metrics analysis. Such features filtering strategy (with the cut off of 10) was adopted from the --p-min-reads default parameters in the [qiime2 deblur workflow](https://docs.qiime2.org/2018.2/plugins/available/deblur/denoise-16S/), which retains only features appearing at least min_reads 10 times across all samples in the resulting feature-table by default.

In [ ]:
#filter features with frequency less than 10
!qiime feature-table filter-features \
  --i-table dada2_malesrt/malesrt2_table-dada2.qza \
  --p-min-frequency 10 \
  --o-filtered-table dada2_malesrt/malesrt2_table-dada2_filtered10-table.qza

In [ ]:
#summarize filtered feature-table
!qiime feature-table summarize \
  --i-table dada2_malesrt/malesrt2_table-dada2_filtered10-table.qza \
  --o-visualization dada2_malesrt/malesrt2_table-dada2_filtered10-table.qzv \
  --m-sample-metadata-file metadata_malesrt.txt

## rarefaction

Examine sequencing depth across samples that were processed using dada2 (and filtered), and choose the even sampling depth. The maximum amount of reads per sample in the data set is 40855. This upper limit will be used to generate alpha rarefaction curves to examine sequencing coverage.

In [ ]:
#make alpha-rarefaction plot and estimate community coverage
!qiime diversity alpha-rarefaction \
 --i-table dada2_malesrt/malesrt2_table-dada2_filtered10-table.qza \
 --i-phylogeny dada2_malesrt/rooted-malesrt2_tree_dada2.qza \
 --p-max-depth 40855 \
 --p-metrics faith_pd \
 --p-metrics shannon \
 --p-metrics observed_otus \
 --p-metrics chao1 \
 --m-metadata-file metadata_malesrt.txt \
 --o-visualization dada2_malesrt/alpha-rare_malesrt2_filtered10_dada2.qzv

Based on the alpha-rarefaction and the table generated by dada2, we would rarefy the data set at the 14981 (minimum sample frequency) reads per sample. This is because the rarefaction curve level off well before this value and with this even sampling depth we likely sample most of the community, while analysing all the samples.

In [ ]:
#run core-metrics pipeline at a given sequencing depth (rarefied data)
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny dada2_malesrt/rooted-malesrt2_tree_dada2.qza \
  --i-table dada2_malesrt/malesrt2_table-dada2_filtered10-table.qza \
  --p-sampling-depth 14981 \
  --m-metadata-file metadata_malesrt.txt \
  --output-dir dada2_malesrt/core-metrics-dada2_results_filtered10/

## taxonomic analysis

Upload the GreenGenes 'reference data base'. This is a pre-trained Naive Bayes classifier for the use with the q2-feature-classifier plugin. This classifier was trained on the Greengenes 13_8 99% OTUs, where the sequences have been trimmed to only include 250 bases from the region of the 16S that was sequenced in this study (the V4 region, bound by the 515F/806R EMP primer pair). See more information of the pre-trained classifiers [here](https://docs.qiime2.org/2018.2/data-resources/)

In [ ]:
!wget -O "gg-13-8-99-515-806-nb-classifier.qza" "https://data.qiime2.org/2018.2/common/gg-13-8-99-515-806-nb-classifier.qza"

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads dada2_malesrt/malesrt2_rep-seqs-dada2.qza \
  --o-classification dada2_malesrt/malesrt2_dada2_taxonomy.qza

!qiime metadata tabulate \
  --m-input-file dada2_malesrt/malesrt2_dada2_taxonomy.qza \
  --o-visualization dada2_malesrt/malesrt2_dada2_taxonomy.qzv

Examine the microbiota taxonomic composition in the CMR samples with the interactive bar plots. Rarefaction procedure in the core-metric pipeline above, sampling min reads at random. Thus, for the complete match in the data comparison, examine the rarefied ('rarefied_table.qza') feature-table found in [this repository at GitHub](https://github.com/alavrinienko/chern-cmr-voles).

In [ ]:
#create the taxa bar plots using the 'filtered10' and rarefied at 14981 reads/sample feature-table
!qiime taxa barplot \
  --i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
  --i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
  --m-metadata-file metadata_malesrt.txt \
  --o-visualization dada2_malesrt/malesrt2_filtered10_even14981_dada2_taxa-bar-plots.qzv

## dada2 relative frequency tables

Here we generate the tables with relative abundance data at several taxanomical levels (L2 phylum, L3 class, L4 order, L5 family, L6 genus, L7 species). These are based on the 'filtered10' and rarefied at 14981 reads/sample feature-table.

In [ ]:
!qiime taxa collapse \
--i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
--i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
--p-level 2 \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L2-filtered10-even-dada2

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2_malesrt/taxa-relative-filtered10-even-dada2/L2-filtered10-even-dada2/collapsedL2_table.qza \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L2-filtered10-even-dada2/relative-L2-filtered10-even-dada2

In [ ]:
!qiime taxa collapse \
--i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
--i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
--p-level 3 \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L3-filtered10-even-dada2

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2_malesrt/taxa-relative-filtered10-even-dada2/L3-filtered10-even-dada2/collapsedL3_table.qza \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L3-filtered10-even-dada2/relative-L3-filtered10-even-dada2

In [ ]:
!qiime taxa collapse \
--i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
--i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
--p-level 4 \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L4-filtered10-even-dada2

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2_malesrt/taxa-relative-filtered10-even-dada2/L4-filtered10-even-dada2/collapsedL4_table.qza \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L4-filtered10-even-dada2/relative-L4-filtered10-even-dada2

In [ ]:
!qiime taxa collapse \
--i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
--i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
--p-level 5 \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L5-filtered10-even-dada2

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2_malesrt/taxa-relative-filtered10-even-dada2/L5-filtered10-even-dada2/collapsedL5_table.qza \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L5-filtered10-even-dada2/relative-L5-filtered10-even-dada2

In [ ]:
!qiime taxa collapse \
--i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
--i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
--p-level 6 \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L6-filtered10-even-dada2

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2_malesrt/taxa-relative-filtered10-even-dada2/L6-filtered10-even-dada2/collapsedL6_table.qza \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L6-filtered10-even-dada2/relative-L6-filtered10-even-dada2

In [ ]:
!qiime taxa collapse \
--i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
--i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
--p-level 7 \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L7-filtered10-even-dada2

In [ ]:
!qiime feature-table relative-frequency \
--i-table dada2_malesrt/taxa-relative-filtered10-even-dada2/L7-filtered10-even-dada2/collapsedL7_table.qza \
--output-dir dada2_malesrt/taxa-relative-filtered10-even-dada2/L7-filtered10-even-dada2/relative-L7-filtered10-even-dada2

We converted the biom tables to tsv prior merging it all into one file containing the taxonomy and the metadata associated with each bank vole individual (this can be done using the qiime/1.9.1 version). We then calculated means and SD for each taxa per the CMR-group, and archive these data for the manuscript (see the ESM table S2).

In [ ]:
#open relative-frequency_table.qza with any zip archive app and export biom table,
#then, run the following:
!module load qiime/1.9.1
!biom convert -i feature-tableL2.biom -o L2-filtered10-even-dada2/relative-L2-filtered10-even-dada2/feature-tableL2.tsv --to-tsv
!biom convert -i feature-tableL3.biom -o L3-filtered10-even-dada2/relative-L3-filtered10-even-dada2/feature-tableL3.tsv --to-tsv
!biom convert -i feature-tableL4.biom -o L4-filtered10-even-dada2/relative-L4-filtered10-even-dada2/feature-tableL4.tsv --to-tsv
!biom convert -i feature-tableL5.biom -o L5-filtered10-even-dada2/relative-L5-filtered10-even-dada2/feature-tableL5.tsv --to-tsv
!biom convert -i feature-tableL6.biom -o L6-filtered10-even-dada2/relative-L6-filtered10-even-dada2/feature-tableL6.tsv --to-tsv
!biom convert -i feature-tableL7.biom -o L7-filtered10-even-dada2/relative-L7-filtered10-even-dada2/feature-tableL7.tsv --to-tsv

## core features analysis

We will filter the dada2 (filtered10, rarefied) feature-table to include only samples from (1) CL (radioactively uncontaminated) and (2) CH (radioactively contaminated) areas within the Chernobyl Exclusion Zone, Ukraine. These tables will be used (with the core-features plugin) to identify potential differences in the core microbiome between treatments.

In [ ]:
#CL (uncontaminated) CMR individuals - to retain only samples in group 'CL'
!qiime feature-table filter-samples \
  --i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
  --m-metadata-file metadata_malesrt.txt \
  --p-where "Treatment='CL'" \
  --o-filtered-table dada2_malesrt/filtered10-even-select-tables/clean-dada2-filtered10-even-table.qza
  
#visualize and double-check the filtering
!qiime feature-table summarize \
  --i-table dada2_malesrt/filtered10-even-select-tables/clean-dada2-filtered10-even-table.qza \
  --o-visualization dada2_malesrt/filtered10-even-select-tables/clean-dada2-filtered10-even-table.qzv \
  --m-sample-metadata-file metadata_malesrt.txt

In [ ]:
#CH (contaminated) CMR individuals - to retain only samples in group 'CH'
!qiime feature-table filter-samples \
  --i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
  --m-metadata-file metadata_malesrt.txt \
  --p-where "Treatment='CH'" \
  --o-filtered-table dada2_malesrt/filtered10-even-select-tables/hot-dada2-filtered10-even-table.qza
  
#visualize and double-check the filtering
!qiime feature-table summarize \
  --i-table dada2_malesrt/filtered10-even-select-tables/hot-dada2-filtered10-even-table.qza \
  --o-visualization dada2_malesrt/filtered10-even-select-tables/hot-dada2-filtered10-even-table.qzv \
  --m-sample-metadata-file metadata_malesrt.txt

Run the core-features analysis on the filtered datasets (CL/CH). Examine the output files and archive these data for the manuscript (see the ESM table S5).

In [ ]:
!qiime feature-table core-features \
  --i-table dada2_malesrt/filtered10-even-select-tables/clean-dada2-filtered10-even-table.qza \
  --p-min-fraction 0.9 \
  --p-max-fraction 1.0 \
  --p-steps 5 \
  --output-dir dada2_malesrt/core-features-filtered10-even-dada2/core-even-clean/

In [ ]:
!qiime feature-table core-features \
  --i-table dada2_malesrt/filtered10-even-select-tables/hot-dada2-filtered10-even-table.qza \
  --p-min-fraction 0.9 \
  --p-max-fraction 1.0 \
  --p-steps 5 \
  --output-dir dada2_malesrt/core-features-filtered10-even-dada2/core-even-hot/

## alpha diversity analysis

We have examined differences in the alpha diversity (number of observed ASVs, Shannon Index) between (CL vs. CH), and also within (first vs. second) treatment groups using the Kruskal-Wallis tests implemented in the DUNN.TEST package in R. All comparisons were not significant (see the ESM table S3). The input data for these comparisons derived from the *observed_otus_vector.qza* and the *shannon_vector.qza* files from the core diversity metrics pipeline (described above).

For the visual purposes, we would also want to examine changes in the alpha diversity between the first and second capture of CL and CH bank voles, in a paired mode using the qiime2 [longitudinal plugin](https://docs.qiime2.org/2018.2/plugins/available/longitudinal/pairwise-differences/).

In [ ]:
!qiime longitudinal pairwise-differences \
  --m-metadata-file metadata_malesrt.txt \
  --m-metadata-file dada2_malesrt/core-metrics-dada2_results_filtered10/observed_otus_vector.qza \
  --p-metric observed_otus \
  --p-group-column Treatment \
  --p-state-column PrePost \
  --p-state-1 1 \
  --p-state-2 2 \
  --p-individual-id-column StudyID \
  --p-replicate-handling random \
  --o-visualization malesrt-q2-observed_pairwise-differences.qzv

In [ ]:
!qiime longitudinal pairwise-differences \
  --m-metadata-file metadata_malesrt.txt \
  --m-metadata-file dada2_malesrt/core-metrics-dada2_results_filtered10/shannon_vector.qza \
  --p-metric shannon \
  --p-group-column Treatment \
  --p-state-column PrePost \
  --p-state-1 1 \
  --p-state-2 2 \
  --p-individual-id-column StudyID \
  --p-replicate-handling random \
  --o-visualization malesrt-q2-shannon_pairwise-differences.qzv

## beta diversity

Here we will use the pairwise permanova analysis, to test whether the beta diversity is significantly different between the first and second captures within/and also between bank voles inhabiting CL and CH areas. Examine the output files and archive these data for the manuscript (see the ESM table S6).

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix dada2_malesrt/core-metrics-dada2_results_filtered10/bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata_malesrt_SIA_f.txt \
  --m-metadata-column PrePostCLCH_f \
  --o-visualization bray-PrePostCLCH-significance.qzv \
  --p-pairwise

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix dada2_malesrt/core-metrics-dada2_results_filtered10/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata_malesrt_SIA_f.txt \
  --m-metadata-column PrePostCLCH_f \
  --o-visualization unwUniFrac-PrePostCLCH-significance.qzv \
  --p-pairwise

We would also like to know which factors are responsible for the patterns observed in the beta diversity ordinations (including numerical data categories). We will use the adonis function in the vegan R package implemented in qiime/1.9.1. The input distance files for the adonis tests were extracted from the bray and unwUF 'distance matrix' files from the core diversity metrics pipeline (described above). Examine the output files and archive these data for the manuscript (see the ESM table S6).

In [ ]:
#bray-curtis
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c Treatment -o adonis_out-bray-treatment -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c PrePostCLCH_f -o adonis_out-bray-prepostclch_f -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c BodyMass -o adonis_out-bray-bodymass -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c HeadWidth -o adonis_out-bray-headwidth -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c BCI -o adonis_out-bray-bci -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c ExternalDoseTLD -o adonis_out-bray-externaldosetld -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c InternalDose -o adonis_out-bray-internaldose -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c TotalDose -o adonis_out-bray-totaldose -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c corrLiverd13C -o adonis_out-bray-corrliver13c -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c corrLiverd15N -o adonis_out-bray-corrliver15n -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c InTheLab -o adonis_out-bray-labdays -n 999
!compare_categories.py --method adonis -i bray-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt.txt -c InTheField -o adonis_out-bray-fielddays -n 999

#unweighted UniFrac
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c Treatment -o adonis_out-unwuf-treatment -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c PrePostCLCH_f -o adonis_out-unwuf-prepostclch_f -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c BodyMass -o adonis_out-unwuf-bodymass -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c HeadWidth -o adonis_out-unwuf-headwidth -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c BCI -o adonis_out-unwuf-bci -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c ExternalDoseTLD -o adonis_out-unwuf-extarnaldosetld -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c InternalDose -o adonis_out-unwuf-internaldose -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c TotalDose -o adonis_out-unwuf-totaldose -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c corrLiverd13C -o adonis_out-unwuf-corrliver13c -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c corrLiverd15N -o adonis_out-unwuf-corrliver15n -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c InTheLab -o adonis_out-unwuf-labdays -n 999
!compare_categories.py --method adonis -i unwUF-distance-matrix-malesrt-filtered10.txt -m metadata-malesrt-all-f.txt -c InTheField -o adonis_out-unwuf-fielddays -n 999

## ordination

Results of the PCoA were examined using the Emperor 3D plots from the core diversity metrics pipeline (described above). However, we decided also to use a stand alone Emperor software for higher flexibility and to utilise 'add-vectors' function, which is apparently not available through the qiime2 Emperor plugin. This function connects samples from multiple observations of the same subject with a solid line. The ordination-bray/unwUF.txt files are coming from the *pcoa_results.qza* file (bray/unwUF, respectively).

In [ ]:
!module load qiime/1.9.1
!make_emperor.py -i ordination-bray.txt -m metadata_malesrt.txt -o vectors-bray --add_vectors SubjectID
!make_emperor.py -i ordination-unwUF.txt -m metadata_malesrt.txt -o vectors-unwUF --add_vectors SubjectID

We have also used qiime 1.9.1 to generate the [2D plots](https://forum.qiime2.org/t/is-it-available-to-make-2d-pcoa-plots/726). Then, these vector graphics was edited for the manuscript figure 2a.

In [ ]:
!module load qiime/1.9.1
!make_2d_plots.py -i ordination-bray.txt -m metadata_malesrt.txt -b 'CMR-group'
!make_2d_plots.py -i ordination-unwUF.txt -m metadata_malesrt.txt -b 'CMR-group'

Extract the PCoA axis1 and run pairwise comparisons between the first and second samples collected from bank voles inhabiting CL and CH areas.

In [ ]:
!qiime longitudinal pairwise-differences \
  --m-metadata-file metadata_malesrt.txt \
  --m-metadata-file dada2_malesrt/core-metrics-dada2_results_filtered10/bray_curtis_pcoa_results.qza \
  --p-metric 'Axis 1' \
  --p-group-column Treatment \
  --p-state-column PrePost \
  --p-state-1 1 \
  --p-state-2 2 \
  --p-individual-id-column StudyID \
  --p-replicate-handling random \
  --o-visualization malesrt-q2-brayaxis1_pairwise-differences.qzv

In [ ]:
!qiime longitudinal pairwise-differences \
  --m-metadata-file metadata_malesrt.txt \
  --m-metadata-file dada2_malesrt/core-metrics-dada2_results_filtered10/unweighted_unifrac_pcoa_results.qza \
  --p-metric 'Axis 1' \
  --p-group-column Treatment \
  --p-state-column PrePost \
  --p-state-1 1 \
  --p-state-2 2 \
  --p-individual-id-column StudyID \
  --p-replicate-handling random \
  --o-visualization malesrt-q2-unwUFaxis1_pairwise-differences.qzv

## ds-fdr differential abundance testing

We will run [ds-fdr plugin](https://forum.qiime2.org/t/q2-dsfdr-community-tutorial/5559) within the qiime2 to identify differentially abundant features within the CMR dataset, based on the treatment or timepoint/treatment categories. For this, we would have to first filter feature-table to include only part of the samples that represent specific categories of interest, since the ds-fdr method can test differences between two groups only.

We will filter the dada2 (filtered10, rarefied) feature-table to include only samples from (1) 'first' and (2) 'second' capture during the CMR (see the PrePost_f metadata column). Also, we will make use of filtered tables that include only samples from (3) CL (radioactively uncontaminated) and (4) CH (radioactively contaminated) areas within the Chernobyl Exclusion Zone, Ukraine. These tables were filtered during the preparation for the core features analysis described above.

In [ ]:
#retain only samples in group 'first' from the column PrePost_f
!qiime feature-table filter-samples \
  --i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
  --m-metadata-file metadata-malesrt.txt \
  --p-where "PrePost_f='first'" \
  --o-filtered-table dada2_malesrt/filtered10-even-select-tables/first-dada2-filtered10-even-table.qza
  
#visualize and double-check the filtering
!qiime feature-table summarize \
  --i-table dada2_malesrt/filtered10-even-select-tables/first-dada2-filtered10-even-table.qza \
  --o-visualization dada2_malesrt/filtered10-even-select-tables/first-dada2-filtered10-even-table.qzv \
  --m-sample-metadata-file metadata-malesrt.txt

In [ ]:
#retain only samples in group 'second' from the column PrePost_f
!qiime feature-table filter-samples \
  --i-table dada2_malesrt/core-metrics-dada2_results_filtered10/rarefied_table.qza \
  --m-metadata-file metadata-malesrt.txt \
  --p-where "PrePost_f='second'" \
  --o-filtered-table dada2_malesrt/filtered10-even-select-tables/second-dada2-filtered10-even-table.qza
  
#visualize and double-check the filtering
!qiime feature-table summarize \
  --i-table dada2_malesrt/filtered10-even-select-tables/second-dada2-filtered10-even-table.qza \
  --o-visualization dada2_malesrt/filtered10-even-select-tables/second-dada2-filtered10-even-table.qzv \
  --m-sample-metadata-file metadata-malesrt.txt

Note that prior to run the ds-fdr analysis we first need to install the ds-fdr as a qiime2 dev plugin via pip install (see more details [here](https://forum.qiime2.org/t/q2-dsfdr-community-tutorial/5559)). Note, this was run with the qiime2 version change from 2018.2 to 2018.11.

In [ ]:
!qiime dsfdr permutation-fdr --i-table first-dada2-filtered10-even-table.qza --m-metadata-file metadata-malesrt-pre-f.txt --m-metadata-column 'Treatment' --p-alpha 0.1 --o-visualization malesrt-dada2-filt10-even-first-a01-dsfdr.qzv --verbose
!qiime dsfdr permutation-fdr --i-table clean-dada2-filtered10-even-table.qza --m-metadata-file metadata-malesrt-cl-f.txt --m-metadata-column 'PrePost_f' --p-alpha 0.1 --o-visualization malesrt-dada2-filt10-even-clean-a01-dsfdr.qzv --verbose
!qiime dsfdr permutation-fdr --i-table hot-dada2-filtered10-even-table.qza --m-metadata-file metadata-malesrt-ch-f.txt --m-metadata-column 'PrePost_f' --p-alpha 0.1 --o-visualization malesrt-dada2-filt10-even-hot-a01-dsfdr.qzv --verbose

## balance trees analysis in gneiss

I will run the balance trees analysis for detecting community-wide perturbations based on environmental parameters, using gneiss qiime2 plugin (see more details [here](https://docs.qiime2.org/2018.8/tutorials/gneiss/)). Note, this was run with the qiime2 version change from 2018.2 to 2018.8.

Also, prior to running the balance trees analysis I will apply the [contingency-based](https://docs.qiime2.org/2018.8/tutorials/filtering/) filtering to the original (non-rarefied) dada2 feature-table, and exclude features that are found in less than 5 samples. This is necessary to reduce 'noise' by filtering out low-abundance features and features that are rarely found in the dataset.

In [ ]:
#filter features that are present in less than 5 samples
!qiime feature-table filter-features \
  --i-table dada2_malesrt/malesrt2_table-dada2.qza \
  --p-min-samples 5 \
  --o-filtered-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza
  
#summarize filtered feature-table
!qiime feature-table summarize \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --o-visualization dada2_malesrt/malesrt2_table-dada2_smpl5-table.qzv \
  --m-sample-metadata-file metadata-malesrt.txt

We will first construct trees to define partitions of features that co-occur (*i.e.* balances), based on the unsupervised hierarchical clustering (via Ward’s clustering).

In [ ]:
!qiime gneiss correlation-clustering \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --o-clustering dada2_malesrt/balance-trees-smpl5/hierarchy.qza
  
!qiime gneiss ilr-hierarchical \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/hierarchy.qza \
  --o-balances dada2_malesrt/balance-trees-smpl5/balances.qza

In [ ]:
!qiime gneiss ols-regression \
  --p-formula "PrePostCLCH_f" \
  --i-table dada2_malesrt/balance-trees-smpl5/balances.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/hierarchy.qza \
  --m-metadata-file metadata-malesrt.txt \
  --o-visualization dada2_malesrt/balance-trees-smpl5/regression_summary.qzv

We additionally use the LME instead of the OLS linear regression, to include the StudyID as a random factor (as the observations aren't independent).

In [ ]:
#try also to see how the inclusion of the StudyID AND LME modeling will impact the results
!qiime gneiss lme-regression \
  --p-formula "PrePostCLCH_f" \
  --i-table dada2_malesrt/balance-trees-smpl5/balances.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/hierarchy.qza \
  --m-metadata-file metadata-malesrt.txt \
  --p-groups StudyID \
  --o-visualization dada2_malesrt/balance-trees-smpl5/lme-regression_summary-studyid-test.qzv

While the coefficients and the p-values come in agreement between the regression methods, we would still prefer to use the OLS as it allows to evaluate the effect of each covariate by looking at R2diff. See some discussion details [here](https://forum.qiime2.org/t/clustering-and-regression-analysis-using-q2-gneiss/4211/14).

In [ ]:
!qiime gneiss dendrogram-heatmap \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/hierarchy.qza \
  --m-metadata-file metadata-malesrt.txt \
  --m-metadata-column PrePostCLCH_f \
  --p-color-map viridis \
  --o-visualization dada2_malesrt/balance-trees-smpl5/heatmap-virdis.qzv

In [ ]:
!qiime gneiss balance-taxonomy \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/hierarchy.qza \
  --i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
  --p-taxa-level 4 \
  --p-balance-name 'y0' \
  --m-metadata-file metadata-malesrt.txt \
  --m-metadata-column PrePostCLCH_f \
  --o-visualization dada2_malesrt/balance-trees-smpl5/y0_taxa_summaryL4.qzv

Next, we construct trees to define partitions of features that co-occur (*i.e.* balances), based on the gradient clustering using the absorbed radiation doses: (1) from external radiation (TLD-based) (2) internal radiation, and (3) total radiation exposure for each bank vole individual.

In [ ]:
#external radiation (TLD-based) based balances
!qiime gneiss gradient-clustering \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --m-gradient-file metadata-malesrt.txt \
  --m-gradient-column ExternalDoseTLD \
  --o-clustering dada2_malesrt/balance-trees-smpl5/gradient-hierarchy.qza

In [ ]:
!qiime gneiss ilr-hierarchical \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/gradient-hierarchy.qza \
  --o-balances dada2_malesrt/balance-trees-smpl5/gradient-balances.qza

In [ ]:
!qiime gneiss ols-regression \
  --p-formula "PrePostCLCH_f" \
  --i-table dada2_malesrt/balance-trees-smpl5/gradient-balances.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/gradient-hierarchy.qza \
  --m-metadata-file metadata-malesrt.txt \
  --o-visualization dada2_malesrt/balance-trees-smpl5/gradient-regression_summary.qzv

In [ ]:
!qiime gneiss dendrogram-heatmap \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/gradient-hierarchy.qza \
  --m-metadata-file metadata-malesrt.txt \
  --m-metadata-column PrePostCLCH_f \
  --p-color-map viridis \
  --o-visualization dada2_malesrt/balance-trees-smpl5/gradient-heatmap-virdis.qzv

In [ ]:
!qiime gneiss balance-taxonomy \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/gradient-hierarchy.qza \
  --i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
  --p-taxa-level 4 \
  --p-balance-name 'y4' \
  --m-metadata-file metadata-malesrt.txt \
  --m-metadata-column PrePostCLCH_f \
  --o-visualization dada2_malesrt/balance-trees-smpl5/gradient-y4_taxa_summaryL4.qzv

We will also run the same analysis, but using the internal dose estimates (derived from the gamma spectrometer measurments for each bank vole individual) data for bank voles during the CMR.

In [ ]:
#internal radiation based balances
!qiime gneiss gradient-clustering \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --m-gradient-file metadata-malesrt.txt \
  --m-gradient-column InternalDose \
  --o-clustering dada2_malesrt/balance-trees-smpl5/interndose-gradient-hierarchy.qza

In [ ]:
!qiime gneiss ilr-hierarchical \
--i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
--i-tree dada2_malesrt/balance-trees-smpl5/interndose-gradient-hierarchy.qza \
--o-balances dada2_malesrt/balance-trees-smpl5/interndose-gradient-balances.qza

In [ ]:
!qiime gneiss ols-regression \
--p-formula "PrePostCLCH_f" \
--i-table dada2_malesrt/balance-trees-smpl5/interndose-gradient-balances.qza \
--i-tree dada2_malesrt/balance-trees-smpl5/interndose-gradient-hierarchy.qza \
--m-metadata-file metadata-malesrt.txt \
--o-visualization dada2_malesrt/balance-trees-smpl5/interndose-gradient-regression_summary.qzv

In [ ]:
!qiime gneiss dendrogram-heatmap \
--i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
--i-tree dada2_malesrt/balance-trees-smpl5/interndose-gradient-hierarchy.qza \
--m-metadata-file metadata-malesrt.txt \
--m-metadata-column PrePostCLCH_f \
--p-color-map viridis \
--o-visualization dada2_malesrt/balance-trees-smpl5/interndose-gradient-heatmap-virdis.qzv

In [ ]:
!qiime gneiss balance-taxonomy \
--i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
--i-tree dada2_malesrt/balance-trees-smpl5/interndose-gradient-hierarchy.qza \
--i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
--p-taxa-level 4 \
--p-balance-name 'y4' \
--m-metadata-file metadata-malesrt.txt \
--m-metadata-column PrePostCLCH_f \
--o-visualization dada2_malesrt/balance-trees-smpl5/interndose-gradient-y4_taxa_summaryL4.qzv

Finally, we will also run the same analysis, but using the total radiation dose estimates (derived from the sum of both external and internal radiation exposure estimates for each bank vole individual) data for bank voles during the CMR.

In [ ]:
#total radiation based balances
!qiime gneiss gradient-clustering \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --m-gradient-file metadata-malesrt.txt \
  --m-gradient-column TotalDose \
  --o-clustering dada2_malesrt/balance-trees-smpl5/totdose-gradient-hierarchy.qza

In [ ]:
!qiime gneiss ilr-hierarchical \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/totdose-gradient-hierarchy.qza \
  --o-balances dada2_malesrt/balance-trees-smpl5/totdose-gradient-balances.qza

In [ ]:
!qiime gneiss ols-regression \
  --p-formula "PrePostCLCH_f" \
  --i-table dada2_malesrt/balance-trees-smpl5/totdose-gradient-balances.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/totdose-gradient-hierarchy.qza \
  --m-metadata-file metadata-malesrt.txt \
  --o-visualization dada2_malesrt/balance-trees-smpl5/totdose-gradient-regression_summary.qzv

In [ ]:
!qiime gneiss dendrogram-heatmap \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/totdose-gradient-hierarchy.qza \
  --m-metadata-file metadata-malesrt.txt \
  --m-metadata-column PrePostCLCH_f \
  --p-color-map viridis \
  --o-visualization dada2_malesrt/balance-trees-smpl5/totdose-gradient-heatmap-virdis.qzv

In [ ]:
!qiime gneiss balance-taxonomy \
  --i-table dada2_malesrt/malesrt2_table-dada2_smpl5-table.qza \
  --i-tree dada2_malesrt/balance-trees-smpl5/totdose-gradient-hierarchy.qza \
  --i-taxonomy dada2_malesrt/malesrt2_dada2_taxonomy.qza \
  --p-taxa-level 4 \
  --p-balance-name 'y5' \
  --m-metadata-file metadata-malesrt.txt \
  --m-metadata-column PrePostCLCH_f \
  --o-visualization dada2_malesrt/balance-trees-smpl5/totdose-gradient-y5_taxa_summaryL4.qzv

We then plot and explore the balance trees data remotely in R. We directly unpacked the balances from the resulting *qza* files from the *ILR transform* command, and plot the interesting balances separately in R using density and scatter plots (in the manuscript, see figures 2c,d). The commands below describe how to get the raw balances log ratios out of the biom balance file.

In [ ]:
!biom convert -i y0-feature-table.biom -o y0-feature-table.txt --to-tsv --header-key taxonomy --table-type 'OTU table'
!biom convert -i y5-feature-table.biom -o y5-feature-table.txt --to-tsv --header-key taxonomy --table-type 'OTU table'
!biom convert -i y4-extdoseTLD-feature-table.biom -o y4-extdoseTLD-feature-table.txt --to-tsv --header-key taxonomy --table-type 'OTU table'
!biom convert -i y4-interndose-feature-table.biom -o y4-interndose-feature-table.txt --to-tsv --header-key taxonomy --table-type 'OTU table'